## Bootstrap Aggregation (Bagging)

Bagging is an ensemble technique that is used to improve the predictive power of a single estimator.  It can roughly be thought of as leveraging the [wisdom of the crowds](https://en.wikipedia.org/wiki/Wisdom_of_the_crowd).

### Implementation

The name of this meta-algorithm is very telling, and quite simply it is a technique that uses [bootstrap sampling](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) (simply sampling with replacement) and some method to aggregate the results of each classifier (majority vote).  

![](https://lh3.ggpht.com/-XviqQIhpbkk/UL7oBR60qLI/AAAAAAAADEU/8_z7wF5uWAE/s1600/BaggingCropped.png) 

### Psuedocode

Assume we have `m` datapoints and `t` models/hypotheses ($h_{t}$) in our ensemble.

1. Take `t` bootstrap samples each of size `n` where $n = m$ (only [~2/3 of the data points](http://stats.stackexchange.com/questions/88980/why-on-average-does-each-bootstrap-sample-contain-roughly-two-thirds-of-observat) are in each bootstrap)
2. Train a single classifier on one of the bootstrap samples for each hypothesis $h_{t}$, yielding $t$ models.
3. To predict given a new datapoint $X_{i}$, pass the $X_{i}$ through each of the $t$ models to yield $P_{1}, P_{2}, ... P_{t}$ predictions.
4. Take a majority vote of the $t$ predictions for the final classification of $X_{i}$.  


** For regression, follow steps #1 - 3 above exept for the final prediction simply take an average of the $P_{t}$ predictions rather than majority vote.

![](https://upload.wikimedia.org/wikipedia/en/d/de/Ozone.png)


### Properties

Bagging has some interesting properties:

1. Only used to reduce error due to _Variance_ (but bounded by error of a single estimator).  Usually will never be worse than the single model error.
    * typically each component estimator is high variance (i.e. decision tree).
2. Can be parallelized (trained/predicted in parallel) due to independent of each component estimator.
3. The ensemble itself (meta-algorithm) has very few hyperparameters to tune.

### Proof

The math behind the theory of bagging is quite interesting.  I will show here how the error of an ensemble is typically bounded by the error of a single estimator and will never reduce error more than the error introduced to variance.

![](http://www.blaenkdenum.com/images/notes/machine-learning/bias-variance-tradeoff/bias-variance-learning-curve.png)

#### Diversion (bias-variance decomposition)

Let us take a short diversion to mathematically decompose the bias and variance of a model:

$$ Err(x) = E[(Y - \hat{f}(x))^{2}]$$
$$ = (E[\hat{f}] - f(x))^{2} + E[\hat{f}(x) - E[\hat{f}(x)]]^{2} + \sigma_{e}^{2}$$
$$ = Bias^{2} + Variance + Irreducible Error$$

#### Bagging error

Assume $X_{n}$ random variables each with some $W_{i}$ that corresponds to the true label, and where $W$ is independent of our predictions $Y$.

Also assume we have trained $t$ models represented by $C_{1}, C_{2},..., C_{t}$

The **MSE** error is simply $E[(Y - W)^{2}]$ which we are looking to reduce.

$$E[(Y - W)^{2}] = E[(Y - E[Y] + E[Y] - W)^{2}]$$

$$= E[(Y - E[Y])^{2}] + E[([E[Y] - W)^{2}] + E[2(Y - E[Y])(E[Y] - W)]$$

The first term in the last equation above should look very familiar and is in fact the Variance as seen from the bias-variance decomposition.  

And the last term goes to 0 since W and Y are independent and we can convert the expectation of the product in to the product of the expectations.  

$$And\quad E[Y] - E[E[Y]] = 0$$


Rearranging the equation we can see since the variance has to be $\ge$ 0:
    
$$E[(Y - W)^{2}] = E[(Y - E[Y])^{2}] + E[([E[Y] - W)^{2}]$$

$$E[(Y - W)^{2}] \ge E[([E[Y] - W)^{2}]$$

Now remember that:
    
$$E[Y] \approx \frac{1}{M} \sum_{i = 1}^{M} Y^{(i)}$$

Which is coincidentally how we defined our Bagging decision rule (the average of all our predictions for regression).

And define $Z = \frac{1}{M} \sum_{i = 1}^{M} Y^{(i)}$ be the prediction of our ensemble.

Rewriting the above formulation:

$$E[(Y - W)^{2}] \ge E[(Z - W)^{2}]$$

Meaning that the expected squared loss of an individual estimator $\hat{f}$ is always greater than or equal to the expected squared loss of an ensemble of $\hat{f}$.